In [36]:
# %load assign3.py
#https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
import csv
import numpy as np
from string import maketrans
import string
trainlist=[]
DNAdict = {'A':1,'C':2,'G':3,'T':4}
with open('train.csv','rb') as traincsvFile:
    lines=csv.reader(traincsvFile)
    for line in lines:
        trainlist.append(list(line))

    firstrow = trainlist[0]
    del trainlist[0] #delete first row of train.csv
    print trainlist

    

[['0', 'GTAGCGGCGCATGC', '1'], ['1', 'GCCTGCCACTTCCT', '1'], ['2', 'GAGAGGAGGGGTGT', '1'], ['3', 'GGGCAGGGCCGAGG', '1'], ['4', 'AGACGGGCCGCGCC', '1'], ['5', 'CTAGGCCGGAGCGT', '1'], ['6', 'AGGAGGCGGAGGGA', '1'], ['7', 'GCGTAGGGAGGCGG', '1'], ['8', 'ACCTCTGATCATTC', '1'], ['9', 'AGCAAGAACAGCAC', '1'], ['10', 'AGCCCCCGGAGGGA', '1'], ['11', 'GCCATCGCCAACTT', '1'], ['12', 'CGAAAGATTCGAAA', '1'], ['13', 'AGGAGGGCGGGAGA', '1'], ['14', 'TGGGGCGGAGCCAT', '1'], ['15', 'CCGCTCGTCGGCTG', '1'], ['16', 'GGCCGCGCCCACCA', '1'], ['17', 'GGTGCGGCGAGCGG', '1'], ['18', 'GCGCAGCCTTATCC', '1'], ['19', 'GCGGCCAGATGGGG', '1'], ['20', 'TGGGGGCGTAGCGC', '1'], ['21', 'GCGCGGCGAGGAGA', '1'], ['22', 'GGGAGGCGTGGCGC', '1'], ['23', 'GTCGTGGGCCGTGG', '1'], ['24', 'TTTGCCCTCCCCCT', '1'], ['25', 'GCGGCGGGCCCGGG', '1'], ['26', 'CGAGGGCTGGGCCG', '1'], ['27', 'GTTGGGCCGAAACA', '1'], ['28', 'GCCAGTGGGCACGG', '1'], ['29', 'GATGAAGACGAGGA', '1'], ['30', 'CTCGCCGCAGATAT', '1'], ['31', 'AGTATGGAGAGCGG', '1'], ['32', 'CTGAGCAGG

In [37]:
trainlistrow = [x[1] for x in trainlist]
translatetable = maketrans('ACGT','1234')
print trainlist
for eachrow in trainlist:
     eachrow[0] = string.atoi(eachrow[0])
     # print 'eachrow=' + eachrow[1]
     eachrow[1] = string.atoi(eachrow[1].translate(translatetable))
    # print 'new_eachrow=' + eachrow[1]
     eachrow[2] = string.atoi(eachrow[2])
print trainlist
        # l.append(line)
        # l.remove(l[0])
        # print l

    # X= dataset[:,0:8]


[['0', 'GTAGCGGCGCATGC', '1'], ['1', 'GCCTGCCACTTCCT', '1'], ['2', 'GAGAGGAGGGGTGT', '1'], ['3', 'GGGCAGGGCCGAGG', '1'], ['4', 'AGACGGGCCGCGCC', '1'], ['5', 'CTAGGCCGGAGCGT', '1'], ['6', 'AGGAGGCGGAGGGA', '1'], ['7', 'GCGTAGGGAGGCGG', '1'], ['8', 'ACCTCTGATCATTC', '1'], ['9', 'AGCAAGAACAGCAC', '1'], ['10', 'AGCCCCCGGAGGGA', '1'], ['11', 'GCCATCGCCAACTT', '1'], ['12', 'CGAAAGATTCGAAA', '1'], ['13', 'AGGAGGGCGGGAGA', '1'], ['14', 'TGGGGCGGAGCCAT', '1'], ['15', 'CCGCTCGTCGGCTG', '1'], ['16', 'GGCCGCGCCCACCA', '1'], ['17', 'GGTGCGGCGAGCGG', '1'], ['18', 'GCGCAGCCTTATCC', '1'], ['19', 'GCGGCCAGATGGGG', '1'], ['20', 'TGGGGGCGTAGCGC', '1'], ['21', 'GCGCGGCGAGGAGA', '1'], ['22', 'GGGAGGCGTGGCGC', '1'], ['23', 'GTCGTGGGCCGTGG', '1'], ['24', 'TTTGCCCTCCCCCT', '1'], ['25', 'GCGGCGGGCCCGGG', '1'], ['26', 'CGAGGGCTGGGCCG', '1'], ['27', 'GTTGGGCCGAAACA', '1'], ['28', 'GCCAGTGGGCACGG', '1'], ['29', 'GATGAAGACGAGGA', '1'], ['30', 'CTCGCCGCAGATAT', '1'], ['31', 'AGTATGGAGAGCGG', '1'], ['32', 'CTGAGCAGG